In [ ]:
import numpy as np
import cv2

import asyncio
import nest_asyncio
import websockets
nest_asyncio.apply()

In [ ]:
from ultralytics import YOLO
model = YOLO('../best.pt')
# Define the entry/exit boundary line (e.g., a horizontal line at y=250)
boundary_y = 50
entry_count = 0
exit_count = 0

confidence_threshold = 0.9

previous_positions = {}

In [ ]:
def process_image(byte_data):
    byteArray = bytearray()
    byteArray.extend(byte_data)
    flatNumpyArray = np.array(byteArray)
    img = flatNumpyArray.reshape(100,100)
    return img

In [ ]:
def process_and_track(byte_data):
    global entry_count, exit_count, previous_positions
    
    # Convert byte data to image
    frame = process_image(byte_data)
    
    # Ensure the frame was converted successfully
    if frame is None:
        return
    
    # Track objects in the frame
    results = model.track(source=frame)
    
    # Process the tracking results (e.g., draw bounding boxes)
    current_positions = {}
    
    for result in results:
        for box in result.boxes:
            confidence = box.conf[0].item()

            if confidence < confidence_threshold:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
            object_id = int(box.id[0])  # Assuming 'id' contains the unique ID
            
            # Calculate the center of the bounding box
            center_y = (y1 + y2) // 2
            
            # Store the current position
            current_positions[object_id] = center_y
            
            # Draw the bounding box on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            #cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            #cv2.putText(frame, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Check for entries and exits
    for object_id, center_y in current_positions.items():
        if object_id in previous_positions:
            previous_y = previous_positions[object_id]
            
            # Check if the object has crossed the boundary
            if previous_y <= boundary_y < center_y:
                entry_count += 1
            elif previous_y >= boundary_y > center_y:
                exit_count += 1
    
    # Update previous positions
    previous_positions = current_positions
    
    # Draw the boundary line
    cv2.line(frame, (0, boundary_y), (frame.shape[1], boundary_y), (0, 255, 0), 2)
    
    # Display the counts with smaller font size
    cv2.putText(frame, f'Entries: {entry_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    cv2.putText(frame, f'Exits: {exit_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Resize the frame to 500x500 pixels
    resized_frame = cv2.resize(frame, (500, 500))
    return resized_frame
    # Display the frame (optional, for visualization purposes)
    #cv2.imshow('Tracked Frame', resized_frame)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

In [2]:
def show_image(byte_data):
    byteArray = bytearray()
    byteArray.extend(byte_data)
    flatNumpyArray = np.array(byteArray)
    grayImage = flatNumpyArray.reshape(100,100)
    grayImage = cv2.resize(grayImage, (500,500))
    return grayImage

In [3]:
import asyncio
import nest_asyncio
import websockets
nest_asyncio.apply()

async def handle_data(websocket, path):
    async for message in websocket:
        #img = show_image(message)
        #keep showing the imgs with cv2
        #cv2.imshow('image', img)
        #cv2.waitKey(1)
        img_inferred = process_and_track(message)

        if img_inferred != None:
            cv2.imshow('image', img_inferred)
            cv2.waitKey(1)
        else:
            print('Error processing image')

#192.168.3.3
# para testes locais pegar o IPV4
start_server = websockets.serve(handle_data, "192.168.3.3", 999)

# This ensures the server keeps running until it's explicitly stopped
asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()